In [1]:
import os
import glob
from tqdm.notebook import tqdm
import pandas as pd
import urllib3
import json

In [2]:
DATA_PATH = os.getcwd() + r'\target_data'
print(DATA_PATH)

C:\Users\Kislee\PycharmProjects\Korean_textbook\target_data


In [3]:
files = glob.glob(os.path.join(DATA_PATH, '*.xlsx'))
print(files)

['C:\\Users\\Kislee\\PycharmProjects\\Korean_textbook\\target_data\\result_211108.xlsx']


In [4]:
all_dfs = []
for source in files:
    temp_df = pd.read_excel(source, engine='openpyxl')
    all_dfs.append(temp_df)
df = pd.concat(all_dfs)

In [5]:
df = df[df['등급'].isin([1,2,3,4,5,6])]

In [6]:
len(df)

557

In [7]:
df.head()

,학교명,등급,지문
0,경희대,1,1) 가: 저는 장민입니다.\n 나: 저는 한나 요한손입니다.\n2) 가:...
1,경희대,1,1) 가: 어느 나라 사람입니까\n 나: 저는 러시아 사람입니다.\n2) 가: ...
2,경희대,1,1) 제 친구는 친절해요.\n2) 이 사람은 제 친구 한나예요. 한나는 정말 똑똑해...
3,경희대,1,1) 가: 그게 뭐예요?\n 나: 커피예요.\n2) 가: 이게 한국어로 뭐예요?...
4,경희대,1,1) 가: 서점이 어디에 있어요?\n 나: 2층에 있어요.\n2) 가: 커피숍...


In [8]:
df_beginner = df[df['등급'].isin([1,2])]
df_intermediate = df[df['등급'].isin([3,4])]
df_advanced = df[df['등급'].isin([5,6])]

In [9]:
# 언어 분석 기술 문어/구어 중 한가지만 선택해 사용
# 언어 분석 기술(문어)
#openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU" 
# 언어 분석 기술(구어)
openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU_spoken"
accessKey = "37cdd2da-8145-47be-ac46-caae6b04fdcc"
analysisCode = "ner"

In [10]:
def requestNERfromText(text):
    #요청정보 생성
    requestJson = {
        "access_key": accessKey,
        "argument": {
            "text": text,
            "analysis_code": analysisCode
        }
    }
        
    http = urllib3.PoolManager()
    #API 호출
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8"},
        body=json.dumps(requestJson)
    )
        
    #수신 데이터 변환
    if response.status ==  200:
        result = json.loads(response.data)
    else:
        result = None        
    
    return result

In [11]:
def isOCC(tag):
    if tag == 'CV_OCCUPATION':
        return True
    else:
        return False

def isCountry(tag):
    if tag == 'LCP_COUNTRY':
        return True
    else:
        return False

In [12]:
attribute='지문'

In [13]:
#traversal_df = df_beginner
#traversal_df = df_intermediate
#traversal_df = df_advanced
traversal_df = df

In [14]:
#10000 글자 넘는 지문이 있는지 확인 (etri api에서 10000글자까지 허용)
cnt = 0
for cell in traversal_df[attribute]:
    cnt+=1
    if len(cell) > 10000:
        print(cnt+1, len(cell))

In [15]:
etri_results = []

for cell in tqdm(traversal_df[attribute]):
    try:
        ner_info = requestNERfromText(cell)
        etri_results.append(ner_info)
    except:
        print('NER 정보 수집 실패')
        etri_results.append(None) #원본 데이터와 수일치

  0%|          | 0/557 [00:00<?, ?it/s]

In [16]:
len(etri_results)

557

In [17]:
import pickle

In [18]:
with open('ner_pckl',"wb") as f:
    pickle.dump(etri_results, f)

In [19]:
with open('ner_pckl',"rb") as f:
    loaded_data = pickle.load(f)

In [20]:
len(loaded_data)

557

In [21]:
stop_occupation = ['화','사', '여 자','음악 치료']
stop_country = ['한국','대한민국','북한','조선','고구려','북','고려','고조선','남한','삼국','신라','한','남북한','한국대','남','한중일','한국적','청','미','대〜한민국','남북','남 한국','남 제','로마','프랑크','리','백제']

In [23]:
count_replace_rule = {'남프랑스':'프랑스'
}
def exceptional_country_replace(word):
    return count_replace_rule.get(word, word)

In [24]:
#추가할 컬럼들
occ_columns = [] #직업명
country_columns = [] #국가명

for cell in loaded_data:
    ner_list = []
    if cell != None:
        for sent in cell['return_object']['sentence']:
            for item in sent['NE']:
                ner_list.append((item['text'], item['type']))
    
    occ_in_cell = []
    country_in_cell = []

    for word, tag in ner_list:
        if isOCC(tag):
            refined_word = exceptional_occ_word_replace(word)
            if refined_word not in stop_occupation:
                occ_in_cell.append(refined_word)
        elif isCountry(tag):
            refined_word = exceptional_country_replace(word)
            if refined_word not in stop_country: #제외국가 설정
                country_in_cell.append(refined_word)
    
    occ_in_cell.sort() #정렬
    occ_columns.append(occ_in_cell) 
    country_in_cell.sort()
    country_columns.append(country_in_cell)   

In [25]:
occ_sum = ['/'.join(item) for item in occ_columns]
country_sum = ['/'.join(item) for item in country_columns]

In [26]:
df['직업명'] = occ_sum
df['국가명'] = country_sum

In [27]:
df.to_excel('result_211111.xlsx')

In [28]:
#출현 빈도 확인
#탐색 대상
#초급 df_beginner.index
#중급 df_intermediate.index
#고급 df_advanced.index
#전체 df.index
index_list = df.index
len(index_list)

557

In [29]:
country_dic = {}
for i in index_list:
    for item in country_columns[i]:
        country_dic[item] = country_dic.get(item, 0) + 1

country_freq = []
for key, value in country_dic.items():
    country_freq.append((value, key))
country_freq.sort(reverse=True)
print("출현 국가 수: ", len(country_dic))
print("상위빈도 국가", country_freq)

출현 국가 수:  51
상위빈도 국가 [(130, '중국'), (101, '일본'), (92, '미국'), (48, '베트남'), (30, '프랑스'), (26, '영국'), (25, '태국'), (22, '러시아'), (21, '호주'), (21, '인도'), (21, '독일'), (16, '스페인'), (10, '노르웨이'), (9, '이탈리아'), (9, '그리스'), (7, '필리핀'), (7, '몽골'), (6, '캐나다'), (6, '브라질'), (5, '콜롬비아'), (5, '케냐'), (4, '칠레'), (4, '이집트'), (4, '미얀마'), (3, '투발루'), (3, '이란'), (3, '스웨덴'), (3, '몰디브'), (3, '라오스'), (2, '파라과이'), (2, '코스타리카'), (2, '자메이카'), (2, '예멘'), (2, '아이티'), (2, '스위스'), (2, '부탄'), (2, '말레이시아'), (2, '네팔'), (1, '폴란드'), (1, '파나마'), (1, '터키'), (1, '탄자니아'), (1, '키리바시'), (1, '캄보디아'), (1, '인도네시아'), (1, '우즈베키스탄'), (1, '에티오피아'), (1, '바레인'), (1, '멕시코'), (1, '나이지리아'), (1, '과테말라')]


In [31]:
#코드 종료